# Box Draw
Fred Hohman  
Summer 2016

Code for maximizing the class of a pre-trained image classifer to visualize what a neural network understands a particular class to be.

In [4]:
from __future__ import print_function
from scipy.misc import imsave, imresize
import numpy as np
import time
import os
import sys
import h5py

from keras.layers.core import Dense, Activation
from keras.initializations import normal, identity
from keras.utils import np_utils
from keras.layers import *
from keras.optimizers import SGD, RMSprop
from keras import backend as K

from keras.models import *

from scipy import ndimage

from random import randint

import matplotlib.pyplot as plt
# %matplotlib inline



# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.models import Sequential

Using Theano backend.


In [5]:
from helper import make_noise_img, make_box_img, deprocess_image, normalize, preprocess_image
from helper import noiseify, boxify_top_left, circleify_top_left, boxify_top_left_half

In [6]:
try:
    loop_num = int(sys.argv[1])
    print('loop_num', loop_num)
except ValueError:
    loop_num = 0

In [87]:
# dimensions of the generated pictures for each filter.
img_width = 28
img_height = 28

num_of_pictures = 2500*2    #needs to match same on box-train

### paths
# generated_data = '/home/fredhohman/Desktop/data/binary-net/black-box/generated-data/shapes/' + str(loop_num) + '/'

# json_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/box_no128_plus1.json'
# h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/box_no128_plus1.h5'

json_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_classifier_boxbotr' + str(loop_num) + '.json'
h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_classifier_boxbotr' + str(loop_num) + '.h5'
json_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_snn_10samples.json'
h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_snn_10samples.h5'

mnist_temp_h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_0.h5'
mnist_temp_json_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_0.json'

# home
# mnist_temp_h5_path = '/Users/fredhohman/Github/pnnl-summer-research-notes/output/mnist_box_0.h5'
# mnist_temp_json_path = '/Users/fredhohman/Github/pnnl-summer-research-notes/output/mnist_box_0.json'


###post pnnl

#single output
json_path =       '/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_snn.json'
h5_path =         '/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_snn.h5'

#multiple output
json_path_shape = '/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_snn.json'
h5_path_shape =   '/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shape_snn.h5'
json_path_pos =   '/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_pos_snn.json'
h5_path_pos =     '/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_pos_snn.h5'

In [88]:
from keras.datasets import mnist

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


For single output.

In [92]:
input = Input(batch_shape=(1, 1, img_width, img_height))

#load the model
from keras.models import model_from_json
mnist_box = model_from_json(open(json_path).read()) #working: mnist_temp_json_path
mnist_box.load_weights(h5_path) #working: mnist_temp_h5_path

mnist_box = mnist_box(input)

model = Model(input=input, output=[mnist_box])
# print(model.output)

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad', #adadelta
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_14 (InputLayer)            (1, 1, 28, 28)        0                                            
____________________________________________________________________________________________________
model_3 (Model)                  multiple              119314      input_14[0][0]                   
Total params: 119314
____________________________________________________________________________________________________


For multiple output.

In [89]:
input = Input(batch_shape=(1, 1, img_width, img_height))

#load the model
from keras.models import model_from_json
shape = model_from_json(open(json_path_shape).read())
shape.load_weights(h5_path_shape)
shape = shape(input)

pos = model_from_json(open(json_path_pos).read())
pos.load_weights(h5_path_pos)
pos = pos(input)

model = Model(input=input, output=[shape, pos])
# print(model.output)

model.compile(loss='categorical_crossentropy',
              optimizer='sgd', #adadelta
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (1, 1, 28, 28)        0                                            
____________________________________________________________________________________________________
model_1 (Model)                  multiple              117766      input_11[0][0]                   
____________________________________________________________________________________________________
model_3 (Model)                  multiple              119314      input_11[0][0]                   
Total params: 237080
____________________________________________________________________________________________________


In [19]:
print(model.predict(boxify_top_left(np.copy(X_train[0])[None,])))
print(y_train[0])
# model.layers[-1].output_shape

[array([[  1.12204738e-01,   4.42382065e-04,   3.24448338e-03,
          3.57007887e-03,   1.01887845e-02,   5.57478378e-03,
          1.18427418e-01,   4.82560165e-04,   4.23795637e-03,
          2.20400863e-03,   2.96487194e-03,   1.32777472e-03,
          2.34530568e-02,   8.17916170e-03,   1.41456048e-03,
          1.13035669e-03,   1.58872700e-03,   6.99364305e-01]], dtype=float32), array([[ 0.11741489,  0.60344231,  0.0093964 ,  0.00136827,  0.01460454,
         0.25377357]], dtype=float32)]
5


In [31]:
# model.output[0,1]

In [103]:
img_num = 0

def draw_images(img_num):
    # we build a loss function 
    loss = model.output[0,img_num]
#     loss = model.output[0][0,img_num]+model.output[1][0,1]
    
    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)
    
    # this function returns the loss and grads given the input picture
    iterate = K.function([input, K.learning_phase()], [loss, grads])
    
    # random
    input_img_data = np.copy(X_train[img_num][None,]) # starting with image in X_train
#     input_img_data = np.copy(X_train[15][None,]) # starting with image in X_train
#     input_img_data = ndimage.gaussian_filter(np.random.random((1, 1, img_width, img_height))*1.0, 1)
#     input_img_data = np.random.random((1, 1, img_width, img_height))*0.5
#     input_img_data = np.zeros([1,1,28,28])
#     input_img_data = boxify_top_left_half(np.zeros([1, img_width, img_height]))[None,]
    
    temp_time = time.time()
#     print('Time after initialization:' , temp_time - start_time)
    
    # we run gradient ascent    
    step = 0.005 #0.05
    switched_on = True # should be False for drawing VGG pictures
    NUM_ITERS = 2
    INIT_STEP = 300
    L_PHASE = 0
    DROPOUT_RATE = 0.5
    nsteps = 2
    loss_value = None
    idx = 0
#     for idx in range(NUM_ITERS):
    while loss_value < 0.99:

        if not switched_on:        
            image2 = scipy.misc.imresize(input_img_data[0],2.0).transpose((2,0,1))            
            d,w,h = image2.shape
            m = np.mean(image2[:, (w/2 - img_width/2):(w/2 + img_width/2),
                                  (h/2 - img_height/2):(h/2 + img_height/2)])
            input_img_data[0] = image2[:, (w/2 - img_width/2):(w/2 + img_width/2),
                                        (h/2 - img_height/2):(h/2 + img_height/2)]/m

        for rep in range(0,INIT_STEP):
            for j in range(1,(nsteps+1 + idx)):#/((i+1))):
                
                loss_value, grads_value = iterate([input_img_data,L_PHASE])

                temp = np.copy(grads_value[:, (img_width*(0.5-0.5*j/nsteps)):(img_width*(0.5+0.5*j/nsteps)),
                                              (img_height*(0.5-0.5*j/nsteps)):(img_height*(0.5+0.5*j/nsteps))])
                grads_value[:] = 0.0
                grads_value[:, img_width*(0.5-0.5*j/nsteps):img_width*(0.5+0.5*j/nsteps),
                               img_height*(0.5-0.5*j/nsteps):img_width*(0.5+0.5*j/nsteps)] = temp
                
                input_img_data += grads_value * step

                if loss_value > 0.999: #+1 #for multiple
#                     img = deprocess_image(np.copy(input_img_data[0]))
                    img = 1-input_img_data[0,0]
                    imsave('/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/data/binary-net/mnist-box/testing/' + str(loop_num) + '/box_' + str(img_num) + '.png', img)
                    loss_value, grads_value = iterate([input_img_data, L_PHASE])
                    print('Current loss value:', loss_value,'- Current intensity:', np.mean(input_img_data))
                    
#                     plt.imshow(input_img_data[0,0], cmap='gray')
#                     plt.show()

                    return True    # draw an image
            if INIT_STEP % 300 == 0: 
                print(loss_value)
            
        idx += idx

                
    if loss_value < 0.99:
        print('Current loss value:', loss_value, '- Current intensity:', np.mean(input_img_data))
        print('Did not make it to 0.99')
        return False    # did not draw an image
    

In [104]:
######
#MAIN#
######
while img_num < 20: #change num_of_pictures to draw as many as you want
       
    start_time = time.time()
    print('START image', str(img_num))

    result = draw_images(img_num)
               
    end_time = time.time()
    
    if result == True:
        img_num += 1
            
    print('END image', str(img_num-1) + ":", end_time - start_time)

START image 0
0.00400561187416
0.00644578039646
0.010424811393
0.0168532114476
0.0270905140787
0.0430696681142
0.0673543810844
0.10300347209
0.153065085411
0.219568192959
0.302169382572
0.397125840187
0.497544616461
0.595240592957
0.683252513409
0.757525146008
0.817065298557
0.863042771816
0.897665202618
0.923343360424
0.942242205143
0.956122219563
0.966334700584
0.973883509636
0.979499697685
0.983710825443
0.986895859241
0.989327251911
0.99120092392
0.992658913136
0.993804395199
0.994713068008
0.995440483093
0.996028244495
0.996507287025
0.996901035309
0.997227311134
0.997499763966
0.997729003429
0.997923254967
0.998088955879
0.998231232166
0.998354196548
0.998461008072
0.99855440855
0.998636484146
0.998709022999
0.998773396015
0.998830795288
0.998882234097
0.998928546906
0.998970329762
Current loss value: 0.999008238316 - Current intensity: 0.11106
END image 0: 0.93200802803
START image 1
0.0205878634006
0.0334690846503
0.0539510324597
0.0855997726321
0.132514074445
0.198193773627
0.

/Users/fredhohman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:55: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/fredhohman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:58: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: index out of bounds
Apply node that caused the error: Subtensor{int64, int64}(SoftmaxWithBias.0, Constant{0}, Constant{18})
Toposort index: 39
Inputs types: [TensorType(float32, matrix), Scalar(int64), Scalar(int64)]
Inputs shapes: [(1, 18), (), ()]
Inputs strides: [(72, 4), (), ()]
Inputs values: ['not shown', 0, 18]
Outputs clients: [['output']]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/Users/fredhohman/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 212, in dispatch_shell
    handler(stream, idents, msg)
  File "/Users/fredhohman/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 370, in execute_request
    user_expressions, allow_stdin)
  File "/Users/fredhohman/anaconda/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 175, in do_execute
    shell.run_cell(code, store_history=store_history, silent=silent)
  File "/Users/fredhohman/anaconda/lib/python2.7/site-packages/IP